# Generate FORTRAN static modules with OTI or Multidual algebras.
```
Author: Mauricio Aristizabal
Date last modified: Jan 21 2022
Initial date: Jan 13 2022
Contact: mauriaristi@gmail.com
```

This Notebook lets you generate Fortran modules that implements static versions of the library.

They implement the so called semi-sparse implementation to avoid computational overheads.


In [2]:
import pyoti.core   as coti
from pyoti.fmod_writer import writer

In [4]:
# Write OTIs for 4 variables and first order derivatives.
order   = 1
nImBase = 8+22
w = writer( nImBase, order, mdual=False)
w.write_file(tab = '   ', is_std_matmul=True, elemental_feval= True, real_utils=True)

1


In [ ]:
w.name_imdir

In [ ]:
print(w.write_scalar_function_print( tab="   ", level = 0,mode ='M'))

In [3]:
print(w.write_blocksolver_function(function_name = "INV3x3", is_elemental = False, level = 0, tab = "   ", 
    f_name = "inv3x3", var_type= "O", res_type= "O", separator = ",", matSize='3',
    f_open = "(", f_close = ")", addition = "+",generator = None,    overload = None ))

FUNCTION ONUMM3N2_INV3x3(A)&
   RESULT(RES)
   IMPLICIT NONE
   TYPE(ONUMM3N2) , INTENT(IN) :: A(3,3) 
   TYPE(ONUMM3N2) :: RES(SIZE(A,1),SIZE(A,2)) 

   ! Get real part 
   RES%R=INV3x3(A%R)

   ! Order 1
   RES%E1=-MATMUL(RES%R,(MATMUL(RES%E1,A%R)))
   RES%E2=-MATMUL(RES%R,(MATMUL(RES%E2,A%R)))
   RES%E3=-MATMUL(RES%R,(MATMUL(RES%E3,A%R)))

   ! Order 2
   RES%E11=-MATMUL(RES%R,(MATMUL(RES%E11,A%R)+MATMUL(RES%E1,A%E1)))
   RES%E12=-MATMUL(RES%R,(MATMUL(RES%E12,A%R)+MATMUL(RES%E1,A%E2)+&
          MATMUL(RES%E2,A%E1)))
   RES%E22=-MATMUL(RES%R,(MATMUL(RES%E22,A%R)+MATMUL(RES%E2,A%E2)))
   RES%E13=-MATMUL(RES%R,(MATMUL(RES%E13,A%R)+MATMUL(RES%E1,A%E3)+&
          MATMUL(RES%E3,A%E1)))
   RES%E23=-MATMUL(RES%R,(MATMUL(RES%E23,A%R)+MATMUL(RES%E2,A%E3)+&
          MATMUL(RES%E3,A%E2)))
   RES%E33=-MATMUL(RES%R,(MATMUL(RES%E33,A%R)+MATMUL(RES%E3,A%E3)))

END FUNCTION ONUMM3N2_INV3x3



In [12]:
len(w.mult_res_total[2])

3

In [25]:
for i in range(1,w.order+1):
    
    mult_res = w.mult_res_total[i]
    names = w.name_imdir[i]
    print("")
    
    for j in range(len(mult_res)):
        imdir = names[j]
        multres = mult_res[j]
        print(imdir+" = "+imdir,end="")
        for k in range(1,len(multres)):
            lhs = multres[k][0][2]
            rhs = multres[k][1][2]
            print(" - "+lhs+'*'+rhs,end="")
        # end for
        
        print("")
        
    #
   
#
        
    


E1 = E1 - E1*R
E2 = E2 - E2*R

E11 = E11 - E11*R - E1*E1
E12 = E12 - E12*R - E1*E2 - E2*E1
E22 = E22 - E22*R - E2*E2

E111 = E111 - E111*R - E1*E11 - E11*E1
E112 = E112 - E112*R - E1*E12 - E12*E1 - E2*E11 - E11*E2
E122 = E122 - E122*R - E1*E22 - E22*E1 - E2*E12 - E12*E2
E222 = E222 - E222*R - E2*E22 - E22*E2


In [18]:
w.name_imdir

[['R'], ['E1', 'E2'], ['E11', 'E12', 'E22'], ['E111', 'E112', 'E122', 'E222']]

In [2]:
h = coti.get_dHelp()

In [10]:
coti.np.count_nonzero(h.get_fulldir(3,3)==1)

0

In [7]:
coti.get_deriv_factor(h.get_fulldir(1,3))

2.0

In [8]:
order   = 3
nImBase = order 
w = writer( nImBase, order, mdual=True)

In [9]:
w.name_imdir

[['R'], ['E1', 'E2', 'E3'], ['E12', 'E13', 'E23'], ['E123']]

In [2]:
# Generate all mduals up to order:

max_order = 3
for order in range(1,max_order+1):
    w = writer(order, order, mdual=True)
    w.write_file(tab = '  ', is_std_matmul=True, elemental_feval= True)

In [6]:
w.overloads['LOG']

['MDNUM3_LOG', 'MDNUM3_LOG']

In [4]:
print(w.write_elementary_functions())

ELEMENTAL FUNCTION MDNUM3_TAN(X) RESULT(RES)

 TYPE(MDNUM3), INTENT(IN) :: X
 REAL(DP) :: DER0,DER1,DER2,DER3
 
 DER0 = TAN(X%R)
 DER1 = TAN(X%R)**2 + 1
 DER2 = 2*(TAN(X%R)**2 + 1)*TAN(X%R)
 DER3 = 2*(TAN(X%R)**2 + 1)*(3*TAN(X%R)**2 + 1)

 RES = FEVAL(X,DER0,DER1,DER2,DER3)

END FUNCTION MDNUM3_TAN

ELEMENTAL FUNCTION MDNUM3_COS(X) RESULT(RES)

 TYPE(MDNUM3), INTENT(IN) :: X
 REAL(DP) :: DER0,DER1,DER2,DER3
 
 DER0 = COS(X%R)
 DER1 = -SIN(X%R)
 DER2 = -COS(X%R)
 DER3 = SIN(X%R)

 RES = FEVAL(X,DER0,DER1,DER2,DER3)

END FUNCTION MDNUM3_COS

ELEMENTAL FUNCTION MDNUM3_SIN(X) RESULT(RES)

 TYPE(MDNUM3), INTENT(IN) :: X
 REAL(DP) :: DER0,DER1,DER2,DER3
 
 DER0 = SIN(X%R)
 DER1 = COS(X%R)
 DER2 = -SIN(X%R)
 DER3 = -COS(X%R)

 RES = FEVAL(X,DER0,DER1,DER2,DER3)

END FUNCTION MDNUM3_SIN

ELEMENTAL FUNCTION MDNUM3_ATAN(X) RESULT(RES)

 TYPE(MDNUM3), INTENT(IN) :: X
 REAL(DP) :: DER0,DER1,DER2,DER3
 
 DER0 = ATAN(X%R)
 DER1 = 1D0/(X%R**2 + 1)
 DER2 = -2*X%R/(X%R**2 + 1)**2
 DER3 = 2*(4*X%R**2/(X

In [3]:
print(w.write_scalar_feval_elemental(function_name = "FEVAL", is_elemental = True, level = 0, tab = "    ", 
    f_name = "FUNCTION",  separator = ",", lhs_type= "O", lhs_ptr = False,
    f_open = "(", f_close = ")", addition = " + ", to = False,
    overload = None, write_charact=True ))

ELEMENTAL FUNCTION MDNUM3_FEVAL(X,DER0,DER1,DER2,DER3)&
    RESULT(RES)
    IMPLICIT NONE
    !  Definitions
    REAL(DP) :: FACTOR, COEF
    TYPE(MDNUM3), INTENT(IN)  :: X
    REAL(DP), INTENT(IN)  :: DER0,DER1,DER2,DER3
    TYPE(MDNUM3) :: RES
    TYPE(MDNUM3) :: DX, DX_P

    FACTOR = 1.0_DP
    COEF   = 0.0_DP
    DX     = X
    DX_P   = X

    !  Set real part of deltas zero.
    DX%R = 0.0_DP
    DX_P%R = 0.0_DP

    ! Sets real part
    RES = DER0

    ! Sets order 1
    FACTOR = FACTOR * 1
    COEF = DER1 / FACTOR
    ! RES = RES COEF * DX_P
    ! Order 1
    RES%E1 = RES%E1+COEF*DX_P%E1
    RES%E2 = RES%E2+COEF*DX_P%E2
    RES%E3 = RES%E3+COEF*DX_P%E3
    ! Order 2
    RES%E12 = RES%E12+COEF*DX_P%E12
    RES%E13 = RES%E13+COEF*DX_P%E13
    RES%E23 = RES%E23+COEF*DX_P%E23
    ! Order 3
    RES%E123 = RES%E123+COEF*DX_P%E123
    ! DX_P = DX_P * DX
    ! Order 2
    DX_P%E12 = DX_P%E1*DX%E2+DX_P%E2*DX%E1
    DX_P%E13 = DX_P%E1*DX%E3+DX_P%E3*DX%E1
    DX_P%E23 = DX_P%E2*DX%E3+DX_P

In [1]:
print(w.multiplication_like_function_oo( level = "", f_name = "", lhs_name= "LHS", 
    rhs_name= "RHS", res_name = "LHS", separator = " * ", f_open = "", f_close = "", 
    addition = " + ", lhs_order = 2, rhs_order = 1, comment = False ))

NameError: name 'w' is not defined

In [13]:
print(w.gem_like_function_ro(level = "", f_name = "", a_name= "COEF",
    b_name= "DX_P", c_name= "RES", res_name = "RES", separator = " * ", 
    f_open = "", f_close = "",  addition = " + ", b_order = 3, comment = False ))

! Order 3
RES%E123 = RES%E123 + COEF * DX_P%E123



In [14]:
print(w.feval_function_o( level = "", lhs_name= "X", deriv_name= "DERIVS", res_name = "RES"  ))

! Sets real part
RES = DERIVS(1)

! Sets order 1
FACTOR = FACTOR * 1
COEF = DERIVS(2) / FACTOR
RES = GEM(COEF,DX_P,RES)
DX_P = DX_P*DX

! Sets order 2
FACTOR = FACTOR * 2
COEF = DERIVS(3) / FACTOR
RES = GEM(COEF,DX_P,RES)
DX_P = DX_P*DX

! Sets order 3
FACTOR = FACTOR * 3
COEF = DERIVS(4) / FACTOR
RES = GEM(COEF,DX_P,RES)



In [10]:
w.mult_res_total[1]

[[[[0, 0, 'R'], [0, 1, 'E1']], [[0, 1, 'E1'], [0, 0, 'R']]],
 [[[0, 0, 'R'], [1, 1, 'E2']], [[1, 1, 'E2'], [0, 0, 'R']]]]

In [4]:
print(w.write_cr_matrix_form(tab="   ",val_shape = "M"))

FUNCTION MDNUM1_TO_CR_MAT_M(VAL) RESULT(RES)
   IMPLICIT NONE
   TYPE(MDNUM1), INTENT(IN) :: VAL(:,:)
   REAL(DP) :: RES(NUM_IM_DIR*SIZE(VAL,1),NUM_IM_DIR*SIZE(VAL,2)) 
   INTEGER :: NCOLS=1, NROWS=1

   NCOLS = SIZE(VAL,1)
   NROWS = SIZE(VAL,2)

   ! R x R -> R (1, 1)
   RES(1+NROWS*0:NROWS*1,1+NCOLS*0:NCOLS*1) = VAL%R
   ! R x E1 -> E1 (2, 2)
   RES(1+NROWS*1:NROWS*2,1+NCOLS*1:NCOLS*2) = VAL%R
   ! E1 x R -> E1 (2, 1)
   RES(1+NROWS*1:NROWS*2,1+NCOLS*0:NCOLS*1) = VAL%E1
END FUNCTION MDNUM1_TO_CR_MAT_M



In [10]:
print( w.write_scalar_function(function_name = "SUM", is_elemental = True, level = 0, 
tab = "    ", f_name = "", lhs_type= "O",lhs_shape='M', rhs_type= "R",rhs_shape='S', 
separator = "*", f_open = "",  f_close = "", 
generator = w.multiplication_like_function_oo, overload = "*" ) )

FUNCTION MDNUMM3N3_SUM_OR_MS(LHS,RHS)&
    RESULT(RES)
    IMPLICIT NONE
    TYPE(MDNUMM3N3), INTENT(IN) :: LHS(:,:)
    REAL(DP), INTENT(IN) :: RHS
    TYPE(MDNUMM3N3) :: RES(SIZE(LHS,1),SIZE(LHS,2)) 

    !  Multiplication like function 'LHS*RHS'

    ! Order 0
    RES%R = LHS%R*RHS%R

    ! Order 1
    RES%E1 = LHS%R*RHS%E1 + LHS%E1*RHS%R
    RES%E2 = LHS%R*RHS%E2 + LHS%E2*RHS%R
    RES%E3 = LHS%R*RHS%E3 + LHS%E3*RHS%R

    ! Order 2
    RES%E12 = LHS%R*RHS%E12 + LHS%E12*RHS%R + LHS%E1*RHS%E2 &
            + LHS%E2*RHS%E1
    RES%E13 = LHS%R*RHS%E13 + LHS%E13*RHS%R + LHS%E1*RHS%E3 &
            + LHS%E3*RHS%E1
    RES%E23 = LHS%R*RHS%E23 + LHS%E23*RHS%R + LHS%E2*RHS%E3 &
            + LHS%E3*RHS%E2

    ! Order 3
    RES%E123 = LHS%R*RHS%E123 + LHS%E123*RHS%R + LHS%E1*RHS%E23 &
             + LHS%E23*RHS%E1 + LHS%E2*RHS%E13 + LHS%E13*RHS%E2 &
             + LHS%E3*RHS%E12 + LHS%E12*RHS%E3

END FUNCTION MDNUMM3N3_SUM_OR_MS



In [3]:
print(w.set_type_constants( level = ""))

! Constant imaginary directions.
! Order 1
TYPE(MDNUMM3N3) :: E1 = MDNUMM3N3(0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E2 = MDNUMM3N3(0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E3 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP)
! Order 2
TYPE(MDNUMM3N3) :: E12 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E13 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP,0.0_DP)
TYPE(MDNUMM3N3) :: E23 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP,0.0_DP)
! Order 3
TYPE(MDNUMM3N3) :: E123 = MDNUMM3N3(0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,0.0_DP,1.0_DP)



In [3]:
print(w.write_private(level = "", tab = "  "))

PRIVATE :: DP,DER_R_COS,DER_R_SIN, &
           DER_R_LOG,DER_R_EXP,DER_R_POW



In [3]:
print(w.write_scalar_feval(function_name = "FEVAL", is_elemental = True, level = 0, tab = "    ", 
    f_name = "FUNCTION",  separator = ",", lhs_type= "O", lhs_ptr = False,
    f_open = "(", f_close = ")", addition = " + ", to = False,
    overload = None, write_charact=True ))

ELEMENTAL FUNCTION MDNUMM3N3_FEVAL(X,DERIVS)&
    RESULT(RES)
    IMPLICIT NONE
    !  Definitions
    REAL(DP) :: FACTOR=1, COEF=0.0_DP
    TYPE(MDNUMM3N3), INTENT(IN)  :: X
    TYPE(MDNUMM3N3), INTENT(IN)  :: DERIVS(4)
    TYPE(MDNUMM3N3), INTENT(OUT) :: RES
    TYPE(MDNUMM3N3) :: DX = X, DX_P = X

    !  Set real part of deltas zero.
    DX%R = 0.0_DP
    DX_P%R = 0.0_DP

    ! Sets real part
    RES = DERIVS(1)

    ! Sets order 1
    FACTOR = FACTOR * 1
    COEF = DERIVS(2) / FACTOR
    RES = GEM(COEF,DX_P,RES)
    DX_P = DX_P*DX
    
    ! Sets order 2
    FACTOR = FACTOR * 2
    COEF = DERIVS(3) / FACTOR
    RES = GEM(COEF,DX_P,RES)
    DX_P = DX_P*DX
    
    ! Sets order 3
    FACTOR = FACTOR * 3
    COEF = DERIVS(4) / FACTOR
    RES = GEM(COEF,DX_P,RES)
    
END FUNCTION MDNUMM3N3_FEVAL




In [7]:
print(w.feval_function_o(level = "", lhs_name= "X", deriv_name= "DERIVS", res_name = "RES"  ) ) 

! feval function
RES=0.0_dp
!  Real
RES%R = DERIVS(1)



In [3]:

print(w.gem_like_function_oo( level = "", f_name = "", a_name= "A",
    b_name= "B", c_name= "C", res_name = "RES", separator = "*", 
    f_open = "", f_close = "",  addition = " + " ) ) 

!  General multiplication like function 'A*B + C'

! Order 0
RES%R = C%R + A%R*B%R

! Order 1
RES%E1 = C%E1 + A%R*B%E1 + A%E1*B%R
RES%E2 = C%E2 + A%R*B%E2 + A%E2*B%R
RES%E3 = C%E3 + A%R*B%E3 + A%E3*B%R

! Order 2
RES%E12 = C%E12 + A%R*B%E12 + A%E12*B%R + A%E1*B%E2 &
        + A%E2*B%E1
RES%E13 = C%E13 + A%R*B%E13 + A%E13*B%R + A%E1*B%E3 &
        + A%E3*B%E1
RES%E23 = C%E23 + A%R*B%E23 + A%E23*B%R + A%E2*B%E3 &
        + A%E3*B%E2

! Order 3
RES%E123 = C%E123 + A%R*B%E123 + A%E123*B%R + A%E1*B%E23 &
         + A%E23*B%E1 + A%E2*B%E13 + A%E13*B%E2 &
         + A%E3*B%E12 + A%E12*B%E3



In [4]:
print(w.write_scalar_trivar( function_name = "GEM", is_elemental = True, level = 0, tab = "  ", 
    f_name = "", a_type= "R",  b_type= "O", c_type= "O", 
    separator = "*", f_open = "", f_close = "", addition = " + ",generator = w.gem_like_function_oo, 
    overload = None, write_charact=True ))

ELEMENTAL FUNCTION MDNUMM3N3_GEM_RO(A,B,C)&
  RESULT(RES)
  IMPLICIT NONE
  REAL(DP), INTENT(IN) :: A 
  TYPE(MDNUMM3N3), INTENT(IN) :: B 
  TYPE(MDNUMM3N3), INTENT(IN) :: C 
  TYPE(MDNUMM3N3) :: RES 

  !  General multiplication like function 'A*B + C'

  ! Order 0
  RES%R = C%R + A%R*B%R

  ! Order 1
  RES%E1 = C%E1 + A%R*B%E1 + A%E1*B%R
  RES%E2 = C%E2 + A%R*B%E2 + A%E2*B%R
  RES%E3 = C%E3 + A%R*B%E3 + A%E3*B%R

  ! Order 2
  RES%E12 = C%E12 + A%R*B%E12 + A%E12*B%R + A%E1*B%E2 &
          + A%E2*B%E1
  RES%E13 = C%E13 + A%R*B%E13 + A%E13*B%R + A%E1*B%E3 &
          + A%E3*B%E1
  RES%E23 = C%E23 + A%R*B%E23 + A%E23*B%R + A%E2*B%E3 &
          + A%E3*B%E2

  ! Order 3
  RES%E123 = C%E123 + A%R*B%E123 + A%E123*B%R + A%E1*B%E23 &
           + A%E23*B%E1 + A%E2*B%E13 + A%E13*B%E2 &
           + A%E3*B%E12 + A%E12*B%E3

END FUNCTION MDNUMM3N3_GEM_RO



In [6]:
w.mult_res

[[],
 [[], [], []],
 [[['E1', 'E2'], ['E2', 'E1']],
  [['E1', 'E3'], ['E3', 'E1']],
  [['E2', 'E3'], ['E3', 'E2']]],
 [[['E1', 'E23'],
   ['E23', 'E1'],
   ['E2', 'E13'],
   ['E13', 'E2'],
   ['E3', 'E12'],
   ['E12', 'E3']]]]

In [7]:
w.mult_res_total

[[[[[0, 0, 'R'], [0, 0, 'R']]]],
 [[[[0, 0, 'R'], [0, 1, 'E1']], [[0, 1, 'E1'], [0, 0, 'R']]],
  [[[0, 0, 'R'], [1, 1, 'E2']], [[1, 1, 'E2'], [0, 0, 'R']]],
  [[[0, 0, 'R'], [2, 1, 'E3']], [[2, 1, 'E3'], [0, 0, 'R']]]],
 [[[[0, 0, 'R'], [1, 2, 'E12']],
   [[1, 2, 'E12'], [0, 0, 'R']],
   [[0, 1, 'E1'], [1, 1, 'E2']],
   [[1, 1, 'E2'], [0, 1, 'E1']]],
  [[[0, 0, 'R'], [3, 2, 'E13']],
   [[3, 2, 'E13'], [0, 0, 'R']],
   [[0, 1, 'E1'], [2, 1, 'E3']],
   [[2, 1, 'E3'], [0, 1, 'E1']]],
  [[[0, 0, 'R'], [4, 2, 'E23']],
   [[4, 2, 'E23'], [0, 0, 'R']],
   [[1, 1, 'E2'], [2, 1, 'E3']],
   [[2, 1, 'E3'], [1, 1, 'E2']]]],
 [[[[0, 0, 'R'], [5, 3, 'E123']],
   [[5, 3, 'E123'], [0, 0, 'R']],
   [[0, 1, 'E1'], [4, 2, 'E23']],
   [[4, 2, 'E23'], [0, 1, 'E1']],
   [[1, 1, 'E2'], [3, 2, 'E13']],
   [[3, 2, 'E13'], [1, 1, 'E2']],
   [[2, 1, 'E3'], [1, 2, 'E12']],
   [[1, 2, 'E12'], [2, 1, 'E3']]]]]

In [8]:
w.name_imdir

[['R'], ['E1', 'E2', 'E3'], ['E12', 'E13', 'E23'], ['E123']]

In [3]:
print(w.addition_like_oo(level = "", operator = " + ", lhs_name= "LHS", rhs_name= "RHS", res_name = "RES"))

!  Addition like operation ' + '
!  Real
RES%R = LHS%R + RHS%R
! Order 1
RES%E1 = LHS%E1 + RHS%E1
RES%E2 = LHS%E2 + RHS%E2
RES%E3 = LHS%E3 + RHS%E3
! Order 2
RES%E12 = LHS%E12 + RHS%E12
RES%E13 = LHS%E13 + RHS%E13
RES%E23 = LHS%E23 + RHS%E23
! Order 3
RES%E123 = LHS%E123 + RHS%E123



In [20]:
w.name_imdir

[['R'], ['E1', 'E2'], ['E11', 'E12', 'E22'], ['E111', 'E112', 'E122', 'E222']]